In [5]:
#예제 2.1 토큰화 코드

# 띄어쓰기를 기준으로 토큰화

input_text = "나는 최근 파리 여행을 다녀왔다."
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리
str2idx = {word: i for i, word in enumerate(input_text_list)}
# 아이디 -> 토큰 딕셔너리
idx2str = {i: word for i, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다.']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다.': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다.'}
input_ids:  [0, 1, 2, 3, 4]


In [24]:
#예제 2.2 토큰 아이디에서 벡터로 변환
import torch
import torch.nn as nn

embedding_dim = 16 # 임베딩 차원

# 임베딩 레이어 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 임베딩 레이어에 토큰 아이디를 입력하여 임베딩 벡터를 얻음
input_embeddings = embed_layer(torch.tensor(input_ids))

print("input embeddings: ", input_embeddings)
print("length of input embeddings: ", len(input_embeddings))
print("input embeddings shape", input_embeddings.unsqueeze(0).shape)


input embeddings:  tensor([[-0.0352, -0.6777,  0.7082,  0.6101, -0.0137,  0.1060, -0.9154,  1.7668,
         -0.9481, -1.1275,  0.6446,  0.2600, -2.2988, -0.7216, -1.1145,  1.1686],
        [ 0.6352, -1.0302, -1.4892,  1.3853, -1.0192, -0.3341, -2.1287,  0.6837,
         -0.4787,  0.3914, -0.1916,  0.9009, -0.1794, -2.4746,  0.0125, -0.0097],
        [-0.4592, -0.5429,  0.0072, -1.1805, -0.2225,  1.4122, -0.4943, -0.2075,
         -1.0426,  0.5902,  0.8688,  1.5832, -0.2784, -0.1944,  0.3713, -0.4804],
        [ 1.1532,  1.7369,  1.2984, -0.8345, -0.6610, -0.5150, -1.1488, -0.9103,
          1.1436,  0.7846,  0.0310,  0.5009,  1.3412,  0.5401, -0.0779, -0.4912],
        [ 0.8726, -0.9361,  1.2744,  0.2344, -0.1701,  0.5915, -1.5712,  0.0118,
         -0.4281, -0.3840, -1.2857,  0.2508,  0.7332,  0.3082, -0.0503, -1.5878]],
       grad_fn=<EmbeddingBackward0>)
length of input embeddings:  5
input embeddings shape torch.Size([1, 5, 16])


In [29]:
# 예제 2.3 절대적 위치 인코딩

import torch
import torch.nn as nn

embedding_dim = 16 # 임베딩 차원
max_position = 12 # 최대 토큰 수

embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)

token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings.unsqueeze(0)

input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

In [31]:
print("input ids: ", input_ids)
print("position ids: ", position_ids)

input ids:  [0, 1, 2, 3, 4]
position ids:  tensor([[0, 1, 2, 3, 4]])


In [33]:
# 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, embedding_dim)
weight_k = nn.Linear(embedding_dim, embedding_dim)
weight_v = nn.Linear(embedding_dim, embedding_dim)

querys = weight_q(input_embeddings) 
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

print("querys shape: ", querys.shape)
print("keys shape: ", keys.shape)
print("values shape: ", values.shape)


querys shape:  torch.Size([1, 5, 16])
keys shape:  torch.Size([1, 5, 16])
values shape:  torch.Size([1, 5, 16])


In [39]:
# 예제 2.5 스케일 점곱 방식의 어텐션

from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values):
    dim_k = querys.size(-1)
    # 쿼리와 키의 내적을 구하고 스케일링
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    
    # 소프트맥스 함수를 통해 어텐션 가중치 계산
    weights = F.softmax(scores, dim=-1)

    # 가중치와 값의 곱으로 어텐션 값 계산한다. 입력과 동일한 차원을 가진다.
    outputs = weights @ values
    return outputs


In [40]:
# 예제 2.6 어텐션 연산의 입력과 출력

print("원본 입력 형태:", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 후 입력 형태:", after_attention_embeddings.shape)

원본 입력 형태: torch.Size([1, 5, 16])
어텐션 후 입력 형태: torch.Size([1, 5, 16])


In [50]:
# 2.7 어텐션 연산을 수행하는 AttentionHead 클래스
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        # 헤드 차원
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 선형 층
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        # 키 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        # 값 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim)

    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values)
        )
        return outputs
    
attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)